In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel
import torch
from itertools import cycle

# For BM25
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/asgerkromand/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Generate retrieval corpus

In [22]:
path_adam = '/Users/adamwagnerhoegh/Documents/Legal data/domsdatabasen.retsinformation_newer.json'
path_asger = "/Users/asgerkromand/Library/CloudStorage/OneDrive-UniversityofCopenhagen/0. SDS/1 deep learning and nlp/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json"
path_andreas = '' #missing

# Define a function that can cycle through paths the above paths try them out, and yield the path
def path():
    paths = cycle([path_adam, path_asger, path_andreas])
    for path in paths:
        if path != '':
            try:
                with open(path, 'r') as f:
                    data = json.load(f)
                return data
            except:
                pass
        else:
            raise FileNotFoundError('No path to data found')

retsinfo = path()
    
rag_list = []
idx = 0
for lov in tqdm(retsinfo):
    for kapitel in lov['kapitler']:
        lov_navn = lov['shortName']
        for paragraffer in kapitel['paragraffer']:
            temp_paragraf_dict = {}
            temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
            temp_paragraf_dict['lovnavn'] = lov_navn
            temp_paragraf_list = []
            for styk in paragraffer['stk']:
                temp_paragraf_list.append(styk['tekst'])
            temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
            rag_list.append(temp_paragraf_dict)

with open("rag_list.txt", "w") as file:
    for item in rag_list:
        file.write(f"{item}\n")

100%|██████████| 1637/1637 [00:00<00:00, 21308.13it/s]


## Generate dev set

In [ ]:
# load excel files in dev set folder
import os

dev_set_folder = "devset"

dfs = []
for file in os.listdir(dev_set_folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(dev_set_folder, file))
        dfs.append(df)

# merge all excel
dev_set = pd.concat(dfs, ignore_index=True)

# add csv
rag_batch_1_with_qa = pd.read_csv("devset/rag_batch_1_with_qa.csv", sep=";").iloc[:, 1:].dropna()
rag_batch_1_with_qa.columns = dev_set.columns
dev_set = pd.concat([dev_set, rag_batch_1_with_qa], ignore_index=True)

# output dev set
dev_set.to_csv("devset/dev_set.csv", index=False)

## Vectorize retrieval corpus

### Sparse retrieval TF-IDF

In [29]:
rag_list2 = rag_list

def preprocess(rag_list):
    # extract and preprocess text
    corpus = [item['text'] for item in rag_list]
    corpus = [re.sub('\\s{2,}', ' ', 
                     re.sub('\\W|[0-9]|§', ' ',
                           item.lower())) for item in corpus]

    # remove stopwords
    #nltk.download('punkt')
    stop_words = set(stopwords.words('danish'))
    corpus = [' '.join(word for word in text.split() 
                      if word not in stop_words) for text in tqdm(corpus)]
    
    return corpus

corpus = preprocess(rag_list2)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

100%|██████████| 42593/42593 [00:00<00:00, 61001.76it/s]


### Dense retrieval

In [5]:
## WRITE LATER

## RAG retriever

### Sparse retrieval pipeline for BM25


In [37]:
def sparse_retrieval(question, sparse_matrix, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """

    # preprocess and vectorize question
    question_processed = [re.sub('\\s{2,}', ' ', 
                               re.sub('\\W|[0-9]|§', ' ',
                                     question.lower()))]
    
    # remove stopwords
    stop_words = set(stopwords.words('danish'))
    question_processed = [' '.join(word for word in text.split() 
                                 if word not in stop_words) for text in question_processed]
    
    question_vector = vectorizer.transform(question_processed)

    # sparse retrieval (cosine similarity)
    sparse_retrieval = sparse_matrix.dot(question_vector.T).toarray()

    # get top k paragraphs
    top_k = np.argsort(sparse_retrieval.flatten())[-k:]

    return top_k

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = sparse_retrieval(random_question, X)
for i in top_k:
    print(f'{rag_list2[i]["paragraf_nr"]}: {rag_list2[i]["text"]}')

Hvor og hvornår skal en erhverver af en eller flere ihændehaveraktier, som besidder mindre end 5 pct. af selskabskapitalens stemmerettigheder eller mindre end 5 pct. af selskabskapitalen, registreres? 

§ 80 c.: Bidraget til den obligatoriske pensionsordning beregnes med følgende procentsatser: I 2020 med 0,3 pct. I 2020 med 0,3 pct. I 2021 med 0,6 pct. I 2022 med 0,9 pct. I 2023 med 1,2 pct. I 2024 med 1,5 pct. I 2025 med 1,8 pct. I 2026 med 2,1 pct. I 2027 med 2,4 pct. I 2028 med 2,7 pct. I 2029 med 3,0 pct. Fra 2030 med 3,3 pct.
§ 55.: Enhver, der besidder kapitalandele i et kapitalselskab, skal give meddelelse herom til selskabet, når kapitalandelenes stemmeret udgør mindst 5 pct. af selskabskapitalens stemmerettigheder eller udgør mindst 5 pct. af selskabskapitalen eller kapitalandelenes stemmeret udgør mindst 5 pct. af selskabskapitalens stemmerettigheder eller udgør mindst 5 pct. af selskabskapitalen eller ændring i et allerede meddelt besiddelsesforhold bevirker, at grænserne p

In [ ]:
def bm25_retrieval(question, bm25_model, rag_list, k=3):
    """
    Function that takes a question and returns a list of the most relevant paragraphs based on BM25.
    """
    # Preprocess and tokenize the question
    question_processed = re.sub(r'\s{2,}', ' ', 
                                 re.sub(r'\W|[0-9]|§', ' ', question.lower()))
    stop_words = set(stopwords.words('danish'))
    question_tokens = [word for word in question_processed.split() if word not in stop_words]

    # Get BM25 scores for the query
    scores = bm25_model.get_scores(question_tokens)

    # Get the top k results
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Sort scores in descending order

    # Return the top k paragraphs
    return [(rag_list[i]['paragraf_nr'], rag_list[i]['text'], scores[i]) for i in top_k_indices]

# Example Usage
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(f"Question: {random_question}\n")

# Assuming bm25 is the initialized BM25Okapi model
top_k_results = bm25_retrieval(random_question, bm25, rag_list2, k=3)

# Print top-k results
for paragraf_nr, text, score in top_k_results:
    print(f"{paragraf_nr}: {text} (Score: {score:.2f})")

Question: Hvad skal en virksomhed, som har ansat minimum 10 personer, der gør tjeneste som besætningsmedlemmer på et luftfartøj?

§ 40 c.: I virksomheder med 1-9 ansatte, der gør tjeneste som besætningsmedlemmer på luftfartøj, skal virksomhedens arbejde for arbejdsmiljø under denne tjeneste udføres ved personlig kontakt mellem arbejdsgiveren og de ansatte besætningsmedlemmer. (Score: 36.18)
§ 40 c.: I virksomheder med 1-9 ansatte, der gør tjeneste som besætningsmedlemmer på luftfartøj, skal virksomhedens arbejde for arbejdsmiljø under denne tjeneste udføres ved personlig kontakt mellem arbejdsgiveren og de ansatte besætningsmedlemmer. (Score: 36.18)
§ 40 e.: I virksomheder med 10 ansatte eller derover, som gør tjeneste som besætningsmedlemmer på luftfartøj, oprettes en arbejdsmiljøorganisation. Arbejdsmiljøorganisationen udgøres af højst 4 arbejdsmiljørepræsentanter og et tilsvarende antal repræsentanter for arbejdsgiveren. Arbejdsmiljøorganisationen skal inden for virksomheden være rå

### Create embedding corpus

In [7]:
# creating embedding corpus with the 'KennethTM/bert-base-uncased-danish'

# bert_tokenizer = AutoTokenizer.from_pretrained("KennethTM/bert-base-uncased-danish")
# bert_model = AutoModel.from_pretrained("KennethTM/bert-base-uncased-danish")

# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# cls_embeddings = []

# idx = 0

# for item in tqdm(rag_list):
#     # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
#     try:
#         # tokenize texts
#         input_ids = bert_tokenizer.encode(item['text'], return_tensors='pt')
#         # run through BERT
#         with torch.no_grad():  # disable gradient computation for inference
#             outputs = bert_model(input_ids)
#         # extract cls-token
#         cls_vector = outputs.last_hidden_state[:, 0, :]
#         # add cls-vector to list of embeddings
#         cls_embeddings.append(cls_vector)
#     except:
#         # if error then count errors with this
#         idx += 1

# print(f'{idx} no. of errors')

# # concatenate list into torch tensor
# cls_embeddings_tensor = torch.cat(cls_embeddings, dim=0)

In [30]:
# saving the tensor
#torch.save(cls_embeddings_tensor, '/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_tensor.pt')

In [9]:
# loading the tensor

# cls_embeddings_tensor = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_tensor.pt')

In [12]:
# # creating embedding corpus with the 'vesteinn/DanskBERT'

# tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
# model = AutoModel.from_pretrained("vesteinn/DanskBERT")

# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# cls_embeddings = []

# idx = 0

# for item in tqdm(rag_list):
#     # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
#     try:
#         # tokenize texts
#         input_ids = tokenizer.encode(item['text'], return_tensors='pt')
#         # run through BERT
#         with torch.no_grad():  # disable gradient computation for inference
#             outputs = model(input_ids)
#         # extract cls-token
#         cls_vector = outputs.last_hidden_state[:, 0, :]
#         # add cls-vector to list of embeddings
#         cls_embeddings.append(cls_vector)
#     except:
#         # if error then count errors with this
#         idx += 1

# print(f'{idx} no. of errors')

# # concatenate list into torch tensor
# cls_embeddings_tensor_DanskBERT = torch.cat(cls_embeddings, dim=0)

In [ ]:
# saving the DanskBERT-tensor
#torch.save(cls_embeddings_tensor_DanskBERT, '/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

In [ ]:
# loading the DanskBERT-tensor
#cls_embeddings_tensor_DanskBERT = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

### Dense retrieval pipeline

In [17]:
bert_tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
bert_model = AutoModel.from_pretrained("vesteinn/DanskBERT")

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [150]:
def dense_retrieval(question, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """
    
    # Encode the input sentence
    input_ids = bert_tokenizer.encode(question, return_tensors="pt")  # Encode and add batch dimension
    # Pass the input through the model
    
    with torch.no_grad():  # disable gradient computation for inference
        outputs = bert_model(input_ids)

    # Extract the CLS token representation
    cls_vector = outputs.last_hidden_state[:, 0, :]  # CLS token is at position 0
    
    # sparse retrieval (cosine similarity)
    dense_retrieval = cls_embeddings_tensor_DanskBERT @ torch.transpose(cls_vector, 0, 1)
    
    # get top k paragraphs
    top_k_indices = torch.sort(dense_retrieval, descending=True, dim=0)[1][:k]

    return top_k_indices

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = dense_retrieval(random_question, k=3)
for i in top_k:
    print(f'{rag_list[i]["text"]}')

Bedriftværn, der er etableret i henhold til den lovgivning, der er gældende indtil 1. januar 1993, opretholdes, medmindre at hvad? 

Inden der indgås aftale om køb af en pakkerejse, skal den rejsende gives følgende oplysninger, hvis det er relevant for pakkerejsen, herunder om rejseydelsernes væsentligste kendetegn: Rejsedestination, rejserute, varigheden af og tidspunktet for opholdet og, hvis indkvartering er inkluderet, antallet af inkluderede overnatninger, Rejsedestination, rejserute, varigheden af og tidspunktet for opholdet og, hvis indkvartering er inkluderet, antallet af inkluderede overnatninger, anvendte befordringsmidler og deres kendetegn og kategori, sted, dato og tidspunkt for af- og hjemrejse og oplysninger om steder, hvor der gøres ophold undervejs, med angivelse af disse opholds varighed og transportforbindelser, indkvarteringsstedets beliggenhed, væsentligste kendetegn og kategori i henhold til destinationslandets regler, inkluderede måltider, besøg, udflugter eller 

In [ ]:
# fejl i 6, mangler spørgsmålstegn
# 7 er et dårligt spørgsmål
# 21 er også lidt dårlig
# 35 er ukomplet
# 40 og 41 er de samme spørgsmål

